# Useful functions to build a QAOAAnsatz

In [22]:
# ! pip install qiskit-ibm-runtime
# %pip install qiskit-ibm-runtime
# !python3 -m pip install --upgrade pip
# !pip install ipynb

In [23]:
from __future__ import annotations

from ipynb.fs.full.useful_functions_to_study_an_instance import *
from ipynb.fs.full.useful_functions_for_plotting_and_reading import *

## Initialization

In [24]:
from datetime import datetime
import itertools
import math 
import os
import random
import time

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
import pandas as pd
import seaborn as sns

from qiskit import QuantumCircuit, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.quantum_info import random_statevector, Statevector
from qiskit_aer import Aer

import random

In [25]:
# Pre-defined ansatz circuit, operator class and visualization tools
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution

# Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [26]:
def get_circuit_parameters(subsets, verbose=False):
    """
    Calculate and return circuit parameters based on the given subsets.

    Parameters
    ----------
    subsets : list of sets
        List of subsets representing the problem instance. Each subset corresponds 
        to a node in the graph.
    verbose : bool, optional, default=False
        If True, prints additional details during the execution.

    Returns
    -------
    list_of_intersections : list of lists
        A list of intersections (connections) for each node of the instance. 
        Each element is a list of all the nodes that are connected to a given node.
    num_max_ctrl : int
        Maximum degree of the graph, representing the maximum number of intersections 
        (edges) for any node.
    NUM_ANC : int
        Number of ancillas required to implement the MCMTVChain for the circuit.
    QC_DIM : int
        Total number of qubits required for the circuit, which is the sum of the 
        number of nodes (subsets) and the number of ancillas.
    """

    # Build the graph for the subsets.
    list_of_intersections = build_instance_graph(subsets, verbose=False, draw_graph=False)
    
    # Calculate the maximum degree (num_max_ctrl) as the maximum number of intersections.
    num_max_ctrl = max([len(l) for l in list_of_intersections])
    
    # Calculate the number of ancillas required.
    NUM_ANC = num_max_ctrl - 1
    
    # Total qubits in the quantum circuit.
    QC_DIM = len(subsets) + NUM_ANC

    if verbose:
        print("num_max_ctrl", num_max_ctrl)
        print("NUM_ANC", NUM_ANC)
        print("QC_DIM", QC_DIM)

    return list_of_intersections, num_max_ctrl, NUM_ANC, QC_DIM


def build_cost_circuit(n, instance, k, verbose=False):
    """
    Build the cost Hamiltonian and its corresponding quantum circuit for the given problem instance.

    This function constructs the cost Hamiltonian as described in the paper by Wang et al., 
    which is based on the optimization of a problem using a QAOA-style circuit. 
    It also returns a quantum circuit that implements the Hamiltonian evolution.

    Parameters
    ----------
    n : int
        The dimension of the instance, i.e., the number of qubits used for the problem.
    instance : int
        The index or identifier of the specific instance being solved.
    k : float
        A parameter that defines the relationship between two components of the Hamiltonian.
    verbose : bool, optional, default=False
        If True, prints additional details about the construction of the circuit.

    Returns
    -------
    constant : float
        The constant term (-A + B) of the cost Hamiltonian, which does not affect the optimization process.
    hamiltonian : SparsePauliOp
        The cost Hamiltonian operator that encodes the problem.
    qc_ham : QuantumCircuit
        The quantum circuit that implements the Hamiltonian evolution, parametrized by gamma.
    """

    # Define the instance and subsets for the problem.
    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    
    # Get circuit parameters like the number of qubits.
    _, _, _, QC_DIM = get_circuit_parameters(subsets, verbose=verbose)
    
    # Set the lambda parameters based on the instance dimension and k value.
    l2 = 1 / (n * len(U) - 2)
    l1 = k * n * l2  # l1 / l2 must be equal to k * n

    A = l1 * sum([len(S) for S in subsets]) / 2
    B = l2 * n / 2
    constant = -A + B
    
    # Create Z operators with their respective coefficients.
    coeffs = [(l1 * len(S) / 2 - l2 / 2) for S in subsets]
    Z_operators = [("Z", [i], coeffs[i]) for i in range(n)]
    
    # Build the Hamiltonian as a sparse Pauli operator.
    hamiltonian = SparsePauliOp.from_sparse_list(Z_operators, num_qubits=QC_DIM)
    
    # Print debug information if verbose is enabled.
    if verbose:
        print("A =", A)
        print("B =", B)
        print("constant = -A + B =", constant)
        print("\nhamiltonian:\n", hamiltonian)

    # Create the quantum circuit for the Hamiltonian evolution.
    gamma = Parameter("gamma")
    evo = PauliEvolutionGate(hamiltonian, time=gamma)
    
    qc_ham = QuantumCircuit(QC_DIM)
    qc_ham.append(evo, range(QC_DIM))
    
    qc_ham = qc_ham.decompose(reps=2)
    
    return constant, hamiltonian, qc_ham


## Build the cost operator (circuit)
### Paper di Wang et al.:
The objective is to find the $minimum$ of:
$$ H_P = -\lambda_1\sum_i^n w_i\frac{1-Z_i}{2} +\lambda_2 \sum_i^n\frac{1-Z_i}{2} = $$
$$  =- \lambda_1\sum_i^n \frac{w_i}{2} +\lambda_1\sum_i^n \frac{w_iZ_i}{2} +\lambda_2 \sum_i^n\frac{1}{2} -\lambda_2 \sum_i^n\frac{Z_i}{2} = $$
$$  = -A + \lambda_1\sum_i^n \frac{w_iZ_i}{2} + B - \lambda_2 \sum_i^n\frac{Z_i}{2} = $$
$$  = -A +B +\sum_i^n \frac{\lambda_1w_i -\lambda_2}{2}Z_i  $$
where
$$ A =  \frac{\lambda_1}{2}\sum_i^n w_i      $$ 
$$     B = \frac{ n \lambda_2}{2}  $$

In [27]:
# from itertools import permutations, combinations

from more_itertools import distinct_permutations

from qiskit.circuit.library import PauliEvolutionGate
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit, QuantumRegister

In [28]:
def build_cost_circuit(n, instance, k, verbose=False):
    """
    Build the cost Hamiltonian and its corresponding quantum circuit for the given problem instance.

    This function constructs the cost Hamiltonian as described in the paper by Wang et al., 
    which is based on the optimization of a problem using a QAOA-style circuit. 
    It also returns a quantum circuit that implements the Hamiltonian evolution.

    Parameters
    ----------
    n : int
        The dimension of the instance, i.e., the number of qubits used for the problem.
    instance : int
        The index or identifier of the specific instance being solved.
    k : float
        A parameter that defines the relationship between two components of the Hamiltonian.
    verbose : bool, optional, default=False
        If True, prints additional details about the construction of the circuit.

    Returns
    -------
    constant : float
        The constant term (-A + B) of the cost Hamiltonian, which does not affect the optimization process.
    hamiltonian : SparsePauliOp
        The cost Hamiltonian operator that encodes the problem.
    qc_ham : QuantumCircuit
        The quantum circuit that implements the Hamiltonian evolution, parametrized by gamma.
    """

    # Define the instance and subsets for the problem.
    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    
    # Get circuit parameters like the number of qubits.
    _, _, _, QC_DIM = get_circuit_parameters(subsets, verbose=verbose)
    
    # Set the lambda parameters based on the instance dimension and k value.
    l2 = 1 / (n * len(U) - 2)
    l1 = k * n * l2  # l1 / l2 must be equal to k * n

    A = l1 * sum([len(S) for S in subsets]) / 2
    B = l2 * n / 2
    constant = -A + B
    
    # Create Z operators with their respective coefficients.
    coeffs = [(l1 * len(S) / 2 - l2 / 2) for S in subsets]
    Z_operators = [("Z", [i], coeffs[i]) for i in range(n)]
    
    # Build the Hamiltonian as a sparse Pauli operator.
    hamiltonian = SparsePauliOp.from_sparse_list(Z_operators, num_qubits=QC_DIM)
    
    # Print debug information if verbose is enabled.
    if verbose:
        print("A =", A)
        print("B =", B)
        print("constant = -A + B =", constant)
        print("\nhamiltonian:\n", hamiltonian)

    # Create the quantum circuit for the Hamiltonian evolution.
    gamma = Parameter("gamma")
    evo = PauliEvolutionGate(hamiltonian, time=gamma)
    
    qc_ham = QuantumCircuit(QC_DIM)
    qc_ham.append(evo, range(QC_DIM))
    
    qc_ham = qc_ham.decompose(reps=2)
    
    return constant, hamiltonian, qc_ham

## Build the mixing operator (circuit)

In [29]:
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library.standard_gates import RXGate
from qiskit.circuit import Parameter
from qiskit.circuit.library import MCMTVChain

In [30]:
def build_mixing_circuit(n, instance, verbose=False):
    """
    Build the mixing Hamiltonian quantum circuit for the given problem instance.

    This function constructs a mixing operator for the QAOA-style quantum circuit, which
    is used to explore the solution space by applying rotations to the qubits. It returns
    a quantum circuit that implements the mixing evolution.

    Parameters
    ----------
    n : int
        The dimension of the instance, i.e., the number of qubits used for the problem.
    instance : int
        The index or identifier of the specific instance being solved.
    verbose : bool, optional, default=False
        If True, prints additional details during the circuit construction.

    Returns
    -------
    qc_mixing : QuantumCircuit
        The quantum circuit that implements the mixing Hamiltonian evolution, parametrized by beta.
    """
    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    
    list_of_intersections, num_max_ctrl, NUM_ANC, QC_DIM = get_circuit_parameters(subsets, verbose=verbose)

    #######################################################################
    #######################################################################

    # Initialize the circuit.
    qr = QuantumRegister(n, 'q')
    anc = QuantumRegister(NUM_ANC, 'ancilla')
    qc_mixing = QuantumCircuit(qr, anc)
    
    
    ### Inizializzare le ancille a 1 a ogni strato del QAOA non serve,
    ### basta inizializzarle una volta sola a p=1 perché su ogni ancilla agisce un 
    ### numero pari di NOT-gate in ogni strato.
    ### In realtà uno potrebbe inizializzarle in ogni strato per 
    ### proteggersi da eventuali bit-flip.
    
    # # Initialize ancillas to 1.
    # for ancilla in range(n, QC_DIM):
    #     qc_mixing.initialize(1, ancilla)
    
    
    ### Creo una lista di gate che (tramite VChain) implementano 
    ### X-rotazioni con un diverso numero di controlli. L'elemento i
    ### della lista avrà i+1 controlli.
    
    beta = Parameter('beta')
    g = [MCMTVChain(RXGate(beta), x, 1) for x in range(1, num_max_ctrl+1)]
    gates = [g[i].to_gate() for i in range(len(g))]
    
    
    ### Aggiungo al circuito i gate, specificando quali qubit
    ### devono fare da controlli: ricorda che l'ordine giusto è 
    ### [controlli, target, ancille] quindi se con 5 qubit [0,1,2,3,4] 
    ### e 2 ancille [5,6] voglio fare una rotazione X su 1 
    ### controllata da 0, 2, 3 scriverò:
    ### 
    ### qc_mixing.append(gates[2], [0,2,3, 1, 5,6])
    
    for i, intersections in enumerate(list_of_intersections):
        N = len(intersections)
        qubits_list = intersections + [i] + list(range(n, n+N-1))       
        qc_mixing.append(gates[N-1], qubits_list)
    
    qc_mixing.decompose().draw('mpl')
    
    return qc_mixing



Codice di Qiskit modificato mettendo come controllo "0" invece che "1". C'è anche un esempio da scommentare.

In [31]:
# This code is part of Qiskit.
#
# (C) Copyright IBM 2017, 2020.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""Multiple-Control, Multiple-Target Gate."""


from collections.abc import Callable

from qiskit import circuit
from qiskit.circuit import ControlledGate, Gate, QuantumRegister, QuantumCircuit
from qiskit.exceptions import QiskitError

# pylint: disable=cyclic-import
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate, HGate, TGate, TdgGate, SGate, SdgGate


class MCMT(QuantumCircuit):
    """The multi-controlled multi-target gate, for an arbitrary singly controlled target gate.

    For example, the H gate controlled on 3 qubits and acting on 2 target qubit is represented as:

    .. parsed-literal::

        ───■────
           │
        ───■────
           │
        ───■────
        ┌──┴───┐
        ┤0     ├
        │  2-H │
        ┤1     ├
        └──────┘

    This default implementations requires no ancilla qubits, by broadcasting the target gate
    to the number of target qubits and using Qiskit's generic control routine to control the
    broadcasted target on the control qubits. If ancilla qubits are available, a more efficient
    variant using the so-called V-chain decomposition can be used. This is implemented in
    :class:`~qiskit.circuit.library.MCMTVChain`.
    """

    def __init__(
        self,
        gate: Gate | Callable[[QuantumCircuit, circuit.Qubit, circuit.Qubit], circuit.Instruction],
        num_ctrl_qubits: int,
        num_target_qubits: int,
    ) -> None:
        """Create a new multi-control multi-target gate.

        Args:
            gate: The gate to be applied controlled on the control qubits and applied to the target
                qubits. Can be either a Gate or a circuit method.
                If it is a callable, it will be casted to a Gate.
            num_ctrl_qubits: The number of control qubits.
            num_target_qubits: The number of target qubits.

        Raises:
            AttributeError: If the gate cannot be casted to a controlled gate.
            AttributeError: If the number of controls or targets is 0.
        """
        if num_ctrl_qubits == 0 or num_target_qubits == 0:
            raise AttributeError("Need at least one control and one target qubit.")

        # set the internal properties and determine the number of qubits
        self.gate = self._identify_gate(gate)
        self.num_ctrl_qubits = num_ctrl_qubits
        self.num_target_qubits = num_target_qubits
        self.ctrl_state = "0" * self.num_ctrl_qubits
        
        num_qubits = num_ctrl_qubits + num_target_qubits + self.num_ancilla_qubits

        # initialize the circuit object
        super().__init__(num_qubits, name="mcmt")
        self._label = f"{num_target_qubits}-{self.gate.name.capitalize()}"

        # build the circuit
        self._build()

    def _build(self):
        """Define the MCMT gate without ancillas."""
        if self.num_target_qubits == 1:
            # no broadcasting needed (makes for better circuit diagrams)
            broadcasted_gate = self.gate
        else:
            broadcasted = QuantumCircuit(self.num_target_qubits, name=self._label)
            for target in list(range(self.num_target_qubits)):
                broadcasted.append(self.gate, [target], [])
            broadcasted_gate = broadcasted.to_gate()

        mcmt_gate = broadcasted_gate.control(self.num_ctrl_qubits, ctrl_state= self.ctrl_state)
        self.append(mcmt_gate, self.qubits, [])

    @property
    def num_ancilla_qubits(self):
        """Return the number of ancillas."""
        return 0

    def _identify_gate(self, gate):
        """Case the gate input to a gate."""
        valid_gates = {
            "ch": HGate(),
            "cx": XGate(),
            "cy": YGate(),
            "cz": ZGate(),
            "h": HGate(),
            "s": SGate(),
            "sdg": SdgGate(),
            "x": XGate(),
            "y": YGate(),
            "z": ZGate(),
            "t": TGate(),
            "tdg": TdgGate(),
        }
        if isinstance(gate, ControlledGate):
            base_gate = gate.base_gate
        elif isinstance(gate, Gate):
            if gate.num_qubits != 1:
                raise AttributeError("Base gate must act on one qubit only.")
            base_gate = gate
        elif isinstance(gate, QuantumCircuit):
            if gate.num_qubits != 1:
                raise AttributeError(
                    "The circuit you specified as control gate can only have one qubit!"
                )
            base_gate = gate.to_gate()  # raises error if circuit contains non-unitary instructions
        else:
            if callable(gate):  # identify via name of the passed function
                name = gate.__name__
            elif isinstance(gate, str):
                name = gate
            else:
                raise AttributeError(f"Invalid gate specified: {gate}")
            base_gate = valid_gates[name]

        return base_gate

    def control(self, num_ctrl_qubits=1, label=None, annotated=False):
        ctrl_state = '0' * num_ctrl_qubits
        """Return the controlled version of the MCMT circuit."""
        if not annotated and ctrl_state is None:
            gate = MCMT(self.gate, self.num_ctrl_qubits + num_ctrl_qubits, self.num_target_qubits)
        else:
            gate = super().control(num_ctrl_qubits, label, ctrl_state, annotated=annotated)
            print(ctrl_state, num_ctrl_qubits, "ciao")
        return gate

    def inverse(self, annotated: bool = False):
        """Return the inverse MCMT circuit, which is itself."""
        return MCMT(self.gate, self.num_ctrl_qubits, self.num_target_qubits)


class MCMTVChain(MCMT):
    """The MCMT implementation using the CCX V-chain.

    This implementation requires ancillas but is decomposed into a much shallower circuit
    than the default implementation in :class:`~qiskit.circuit.library.MCMT`.

    **Expanded Circuit:**

    .. plot::

       from qiskit.circuit.library import MCMTVChain, ZGate
       from qiskit.visualization.library import _generate_circuit_library_visualization
       circuit = MCMTVChain(ZGate(), 2, 2)
       _generate_circuit_library_visualization(circuit.decompose())

    **Examples:**

        >>> from qiskit.circuit.library import HGate
        >>> MCMTVChain(HGate(), 3, 2).draw()

        q_0: ──■────────────────────────■──
               │                        │
        q_1: ──■────────────────────────■──
               │                        │
        q_2: ──┼────■──────────────■────┼──
               │    │  ┌───┐       │    │
        q_3: ──┼────┼──┤ H ├───────┼────┼──
               │    │  └─┬─┘┌───┐  │    │
        q_4: ──┼────┼────┼──┤ H ├──┼────┼──
             ┌─┴─┐  │    │  └─┬─┘  │  ┌─┴─┐
        q_5: ┤ X ├──■────┼────┼────■──┤ X ├
             └───┘┌─┴─┐  │    │  ┌─┴─┐└───┘
        q_6: ─────┤ X ├──■────■──┤ X ├─────
                  └───┘          └───┘
    """

    def _build(self):
        """Define the MCMT gate."""
        control_qubits = self.qubits[: self.num_ctrl_qubits]
        target_qubits = self.qubits[
            self.num_ctrl_qubits : self.num_ctrl_qubits + self.num_target_qubits
        ]
        ancilla_qubits = self.qubits[self.num_ctrl_qubits + self.num_target_qubits :]

        if len(ancilla_qubits) > 0:
            master_control = ancilla_qubits[-1]
        else:
            master_control = control_qubits[0]

        self._ccx_v_chain_rule(control_qubits, ancilla_qubits, reverse=False)
        for qubit in target_qubits:
            self.append(self.gate.control(ctrl_state='0'), [master_control, qubit], [])
        self._ccx_v_chain_rule(control_qubits, ancilla_qubits, reverse=True)

    @property
    def num_ancilla_qubits(self):
        """Return the number of ancilla qubits required."""
        return max(0, self.num_ctrl_qubits - 1)

    def _ccx_v_chain_rule(
        self,
        control_qubits: QuantumRegister | list[circuit.Qubit],
        ancilla_qubits: QuantumRegister | list[circuit.Qubit],
        reverse: bool = False        
    ) -> None:
        """Get the rule for the CCX V-chain.

        The CCX V-chain progressively computes the CCX of the control qubits and puts the final
        result in the last ancillary qubit.

        Args:
            control_qubits: The control qubits.
            ancilla_qubits: The ancilla qubits.
            reverse: If True, compute the chain down to the qubit. If False, compute upwards.

        Returns:
            The rule for the (reversed) CCX V-chain.

        Raises:
            QiskitError: If an insufficient number of ancilla qubits was provided.
        """
        
        ctrl_state='0'*len(control_qubits)
        
        if len(ancilla_qubits) == 0:
            return

        if len(ancilla_qubits) < len(control_qubits) - 1:
            raise QiskitError("Insufficient number of ancilla qubits.")

        iterations = list(enumerate(range(2, len(control_qubits))))
        if not reverse:
            self.ccx(control_qubits[0], control_qubits[1], ancilla_qubits[0], ctrl_state='00')
            for i, j in iterations:
                self.ccx(control_qubits[j], ancilla_qubits[i], ancilla_qubits[i + 1], ctrl_state='00')
        else:
            for i, j in reversed(iterations):
                self.ccx(control_qubits[j], ancilla_qubits[i], ancilla_qubits[i + 1], ctrl_state='00')
            self.ccx(control_qubits[0], control_qubits[1], ancilla_qubits[0], ctrl_state='00')

    def inverse(self, annotated: bool = False):
        return MCMTVChain(self.gate, self.num_ctrl_qubits, self.num_target_qubits)

In [32]:
# from qiskit import QuantumCircuit

# num_qubits = 10
# ctrl_qubit = 3

# qr = QuantumRegister(num_qubits, 'q')
# anc = QuantumRegister(8, 'ancilla')
# qc = QuantumCircuit(qr, anc)

# for i, bit in enumerate("0"*num_qubits):
#     qc.initialize(bit, i)

# theta = Parameter('theta') 
# gate = MCMTVChain(RXGate(theta), 9, 1).to_gate() 
# qc.append(gate, [0,1,2,4,5,6,7,8,9, ctrl_qubit, 10,11,12,13,14,15,16,17])
# qc.measure_all()
# qc.decompose().draw('mpl')

###########################################
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# from qiskit.primitives import StatevectorEstimator, StatevectorSampler
# import pandas as pd

# # Generate a pass manager without providing a backend
# pm = generate_preset_pass_manager(optimization_level=3)
# ansatz_isa = pm.run(qc)
# hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)

# estimator = StatevectorEstimator()
# sampler = StatevectorSampler()

# qc = qc.assign_parameters([np.pi]) # ruoto di pi greco attorno a X
# qc_isa = pm.run(qc)
# result = sampler.run([qc_isa], shots=1024).result()
# samp_dist = result[0].data.meas.get_counts()
# samp_dist

## Build initialization circuit

In [33]:
def build_initialization_circuit(n, instance, init_name, verbose=False):
    """
    Builds a quantum circuit for initialization with options for predefined or custom states.
    
    Parameters
    ----------
    n : int
        Number of qubits in the circuit.
    instance : int
        Identifier for the problem instance.
    init_name : str or list of str
        Name of the initialization:
        - "all0": Initializes all qubits to |0⟩.
        - "all1": Initializes all qubits to |1⟩.
        - List of binary strings: Specifies states for superposition.
    verbose : bool, optional
        If True, enables debug outputs. Default is False.
    
    Returns
    -------
    QuantumCircuit
        A quantum circuit initialized as per the specified configuration.
    """
    _, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    _, _, NUM_ANC, QC_DIM = get_circuit_parameters(subsets, verbose=verbose)

    
    ##### CIRCUIT #####
    qr = QuantumRegister(n, 'q')
    anc = QuantumRegister(NUM_ANC, 'ancilla')
    qc_initial = QuantumCircuit(qr, anc)
    
    
    ##### ANCILLAS #####
    #Initialize ancillas to 1.
    for ancilla in range(n, QC_DIM):
        qc_initial.initialize(1, ancilla)
    
    
    ##### QUBITS #####
    
    #### SCELGO GLI STATI CON CUI CREARE LA SOVRAPPOSIZIONE.
    #### RICORDA CHE DEVONO SODDISFARE IL VINCOLO, OVVERO DEVO
    #### SELEZIONARE INSIEMI CHE NON SI INTERSECANO
    # string = '0'*(n-2) + '11'
    # two_one_states = set(["".join(elem) for elem in distinct_permutations(string)])
    # print(two_one_states)
    # init_state = two_one_states + one_one_states
    # init_state = [EC]  + one_one_states

    if init_name == "all1":
        string = '0'*(n-1) + '1'
        one_one_states = ["".join(elem) for elem in distinct_permutations(string)]
        init_state = one_one_states
    
    elif init_name == "all0": 
        init_state = ["0"*n]

    elif isinstance(init_name, list):
        init_state = init_name
    
    
    # print("init_state:\n", init_state, "\ninit_name:", init_name)
    init_state = [x[::-1] for x in init_state] # reverse their order
    # occurrences = [1454, 1353, 37, 36, 34] # found on Leap
    
    
    #### SCELGO I PESI PER LA SOVRAPPOSIZIONE E GLI ASSEGNO AGLI STATI, 
    #### USANDO UN VETTORE DI LUNGHEZZA 2**n.
    # init_state = np.sqrt(1/len(init_state)) * np.ones(len(init_state)) # equal superposition
    # print(init_state)
    
    vec = np.zeros(2**n) # has length 2**n
    j = 0
    for i,nuple in enumerate(bit_gen(n)):
        state = "".join([str(bit) for bit in nuple]) # strings
        if state in init_state:
            vec[i] = 1
            j = j+1
    
    
    #### TRASFORMO IN STATEVECTOR IL VETTORE SCELTO ####
    state = Statevector(vec)
    qc_initial.initialize(state.data, list(range(n)), normalize=True) # set it as initial state for the first n qubits
    
    
    ##### MISURO PER CONTROLLARE CHE LA SOVRAPPOSIZIONE SIA CORRETTA #####
    ##### commenta per usare la sovrapposizione nel seguito! #####
    # from qiskit_aer import Aer
    # from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
    # from qiskit.quantum_info import random_statevector, Statevector
    # from qiskit_aer import Aer
    # qc_initial.measure_all()
    
    # # Let's see the result
    # svsim = Aer.get_backend('aer_simulator')
    # qc_initial.save_statevector()
    # result = svsim.run(qc_initial).result()
    
    # # Print the statevector neatly:
    # final_state = result.get_statevector()
    # plot_histogram(result.get_counts())

    ## from qiskit.visualization import plot_state_qsphere
    ## array_to_latex(final_state, prefix="\\text{Statevector = }")
    ## plot_state_qsphere(state)

    return qc_initial

## Minimization

In [34]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """
    Computes the energy estimate using the provided estimator.
    
    Parameters
    ----------
    params : ndarray
        Array of parameters for the ansatz circuit.
    ansatz : QuantumCircuit
        Parameterized quantum circuit (ansatz).
    hamiltonian : SparsePauliOp
        Hamiltonian operator for which the energy is estimated.
    estimator : EstimatorV2
        Primitive for computing expectation values.
    
    Returns
    -------
    float
        Estimated energy value.
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    cost = result[0].data.evs[0]
    # objective_func_vals.append(cost)
    return cost

In questa versione sotto c'è anche objective_func_vals, che serve a fare il plot delle iterazioni.

In [ ]:
def cost_func_plot(params, ansatz, hamiltonian, estimator):
    """
    Computes the energy estimate and tracks its value for each iteration.
    
    Parameters
    ----------
    params : ndarray
        Array of parameters for the ansatz circuit.
    ansatz : QuantumCircuit
        Parameterized quantum circuit (ansatz).
    hamiltonian : SparsePauliOp
        Hamiltonian operator for which the energy is estimated.
    estimator : EstimatorV2
        Primitive for computing expectation values.
    
    Returns
    -------
    float
        Estimated energy value.
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    cost = result[0].data.evs[0]
    cost_vs_iteration.append(cost)
    
    return cost

In [1]:
def invert_counts(counts):
    """
    Reverses the bit order in the keys of a quantum measurement result dictionary.
    
    Parameters
    ----------
    counts : dict
        Dictionary of measurement results with binary strings as keys.
    
    Returns
    -------
    dict
        A new dictionary with reversed binary keys.
    """
    return {k[::-1]:v for k, v in counts.items()}

## Find the correct gamma bound by looking for the minimum eigenvalue of the cost hamiltonian

In [36]:
def find_gamma_bound(n, instance, k, verbose=False):
    """
    Computes the upper bound for the gamma parameter in quantum optimization.
    
    The bound is derived by identifying the eigenvalue of the cost Hamiltonian with 
    the smallest absolute value.
    
    Parameters
    ----------
    n : int
        Dimension of the instance.
    instance : int
        Identifier of the problem instance.
    k : float
        Scaling factor for the optimization.
    verbose : bool, optional
        If True, enables debug outputs. Default is False.
    
    Returns
    -------
    int
        The upper bound for gamma, rounded up to the nearest integer.
    """
    # Get the instance and its subsets
    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    
    # Scaling factor for l2
    l2 = 1 / (n * len(U) - 2)

    """
    Example: subsets = [[1, 2], [3], [4, 5, 6], [7, 8, 9, 10]]
             subsets_ord = [[3], [1, 2], [4, 5, 6], [7, 8, 9, 10]]
             t = [0, 1, 3, 6, 10]
             
        We need to find the eigenvalue with the minimum absolute value of:
        |f| = l2 * | \sum x_i  -  k * n * \sum w_i*x_i |
        Let's break it down:
        0) \sum x_i = 0 -> min |f| = 0
        1) \sum x_i = 1 -> min |f| = l2 * | 1 - k * n * w_i|
            Here, w_i is the length of the subset that makes the sum closest to 1,
            which is the smallest length -> it is t[1].
        2) \sum x_i = 2 -> min |f| = l2 * | 2 - k * n * \sum w_i|
            \sum w_i is the sum of the two smallest lengths in the instance -> it is t[2].
        3) ...
        In every case the minimum value of |f| is given by l2 * |1-k*n*t[1]|
    """
    ### Sort subsets in increasing order of length.
    how_many_elements = lambda x: len(x)
    subsets_ord = sorted(subsets, key=how_many_elements)
    
    ### Cumulative sum of subset lengths.
    ### At position i, it contains the sum of the lengths up to subset i, 
    ### picking the largest possible subsets.
    cumul_subsets_len = [0]
    for i, s in enumerate(subsets_ord):
        cumul_subsets_len.append(cumul_subsets_len[-1] + len(s))
    
    ### Calculate the minimum of f as described above.
    f_min = []
    for i, cumul in enumerate(cumul_subsets_len):
        f_min.append(l2 * (i - k * n * cumul))
    
    ### It may seem strange because the function should be limited between -1 and 0,
    ### yet f_min contains values lower than -1. However, this is correct,
    ### since we cannot select more than |U| = 12 elements if we want a feasible state,
    ### and at cumul = 12 we correctly reach the value -1.
    
    ### Now, to find gamma_max, we take the smallest absolute value of f_min.
    ### This will always be the value at position 1.
    gamma_max = 2 * np.pi / abs(f_min[1])
    gamma_max = math.ceil(gamma_max)  # Round to the nearest greater integer
    gamma_bound = math.ceil(gamma_max / 2)

    if verbose:
        print(f"gamma bounds -> [0, {2 * gamma_bound}] or [{-gamma_bound}, {gamma_bound}]")

    return gamma_bound


# ESEMPIO

In [37]:
n = 6 # dimension of the problem
instance = 2 # number of the instance chosen
p = 1 # number of layers
h = 1

### Define the instance.
U, subsets_dict = define_instance(n, instance, verbose=False)
subsets = list(subsets_dict.values())
_, _, states_feasible, energies_feasible, EXACT_COVERS = find_spectrum(U, subsets_dict, n, h)
MEC = [state for state in EXACT_COVERS if state.count("1") == min([x.count("1")  for x in EXACT_COVERS])]


### Define the initial state.
one_one_states = ["".join(elem) for elem in distinct_permutations('0'*(n-1) + '1')]
init_name = one_one_states


### Prepare the cost and mixing circuit.
constant, hamiltonian, qc_cost = build_cost_circuit(n, instance, h, verbose=False)
qc_mixing = build_mixing_circuit(n, instance, verbose=False)
qc_initial = build_initialization_circuit(n, instance, init_name, verbose=False)


### Put everything together.
ansatz = QAOAAnsatz(qc_cost, mixer_operator=qc_mixing, initial_state=qc_initial, reps=p, name='my_QAOA_circuit')
# print(ansatz.decompose(reps=2))

['000001', '000010', '000100', '001000', '010000', '100000']
